### LIBRARY INSTALLATION

In [85]:
 !pip install pmdarima
 !pip install skforecast
 !pip install awswrangler
 !pip install holidays

In [86]:
# IMPORTING LIBRARIES
# ======================================================================================
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import pickle
import time
import os
from io import StringIO
import boto3
import contextlib
import re
import matplotlib.pyplot as plt
import awswrangler as wr
import holidays
#plt.style.use('seaborn-darkgrid')
from io import BytesIO
import joblib
# pmdarima
from pmdarima import ARIMA
from pmdarima import auto_arima
# statsmodels
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
# skforecast
from skforecast.Sarimax import Sarimax
from skforecast.ForecasterSarimax import ForecasterSarimax
from skforecast.model_selection_sarimax import backtesting_sarimax
from skforecast.model_selection_sarimax import grid_search_sarimax
from sklearn.metrics import mean_absolute_error
import warnings

### ABT LOAD

In [87]:
# Set up S3 client
client = boto3.client('s3')
bucket_name = 'viamericas-datalake-dev-us-east-1-283731589572-analytics'
path = 'ABTv3_update/ABTv3_update.csv' # Folder under analysis
path2 = 'ABTv3_update'

In [88]:
##Conexion a ABT##
# Listing objects 
elements = client.list_objects(Bucket=bucket_name, Prefix=path)
# Initialize an empty list 
dfs = []
# Iterate through each CSV in the S3 path
for obj in elements.get('Contents', []):
#    print(obj)
    # Check if the object is a CSV file
    if obj['Key'].endswith('.csv'):
        # Read CSV content from S3
        response = client.get_object(Bucket=bucket_name, Key=obj['Key'])
        print(obj['Key'])
        csv_content = response['Body'].read().decode('utf-8')

        # Transform CSV content to df
        df = pd.read_csv(StringIO(csv_content))

        dfs.append(df)
final_df = pd.concat(dfs, ignore_index=True)


ABTv3_update/ABTv3_update.csv


In [89]:
#Date format
final_df['date'] = pd.to_datetime(final_df['date'])

In [90]:
# Create a boolean mask to filter by country and date
mask = (final_df['country'] == 'MEXICO') & (final_df['date'].dt.month == 11) & (final_df['date'].dt.day == 2)
# Mark the Day of the Dead according to the mask
final_df['day_of_the_dead'] = 0  # Initialize with 0
final_df.loc[mask, 'day_of_the_dead'] = 1  # Mark as 1 where the mask is True

In [91]:
#Transformo fecha
final_df['date'] = pd.to_datetime(final_df['date']).dt.date

In [92]:
# Acoto fechas
final_df = final_df[final_df['date'] < pd.to_datetime('2023-12-19').date()]

### PAYER DEFINITION

In [93]:
payer = 'BANCO INDUSTRIAL_GUATEMALA' # 
payer_key = 'BPI'

In [94]:
# Celda de inicio
start_time = time.time()

In [95]:
#### SET UP THE PAYER ####
datos=final_df.loc[final_df.payer_country==payer]
datos = datos.set_index('date')

In [96]:
# datos = final_df[final_df['payer'].str.contains(payer_key)]
# datos = datos.set_index('date')

In [97]:
#final_df[final_df['payer'].str.contains(payer_key)]['date'].min()

### DEFINITION OF VARIABLES TO BE USED

In [98]:
datos = datos.asfreq('D')

In [99]:
#datos.columns.to_list()

In [100]:
datos = datos[['amount', 
                'var_rate_lag_2','var_rate_lag_3','var_rate_lag_4', 'var_rate_lag_5',
                'var_rate_lag_6','var_rate_lag_7',
                'var_rate_lag_8','var_rate_lag_9',
                'var_rate_lag_10','var_rate_lag_11','var_rate_lag_12','var_rate_lag_13', 
                'var_rate_lag_14','var_rate_lag_15','var_rate_lag_16','var_rate_lag_17',
               'var_rate_lag_18','var_rate_lag_19','var_rate_lag_20','var_rate_lag_21',
               'var_rate_lag_22','var_rate_lag_23','var_rate_lag_24','var_rate_lag_25',
                'var_rate_lag_26','var_rate_lag_27','var_rate_lag_28','var_rate_lag_29', 
               'ratio_coupon_tx_lag_2', 
                'ratio_coupon_tx_lag_3', 'ratio_coupon_tx_lag_4', 'ratio_coupon_tx_lag_5', 
                'ratio_coupon_tx_lag_6', 'ratio_coupon_tx_lag_7', 
                'ratio_coupon_tx_lag_8', 
                'ratio_coupon_tx_lag_9', 'ratio_coupon_tx_lag_10', 'ratio_coupon_tx_lag_11', 
                'ratio_coupon_tx_lag_12', 'ratio_coupon_tx_lag_13', 'ratio_coupon_tx_lag_14',
                'ratio_coupon_tx_lag_15','ratio_coupon_tx_lag_16','ratio_coupon_tx_lag_17', 
                'ratio_coupon_tx_lag_18','ratio_coupon_tx_lag_19','ratio_coupon_tx_lag_20',
                'ratio_coupon_tx_lag_21','ratio_coupon_tx_lag_22','ratio_coupon_tx_lag_23',
                'ratio_coupon_tx_lag_24','ratio_coupon_tx_lag_25','ratio_coupon_tx_lag_26',
                'ratio_coupon_tx_lag_27','ratio_coupon_tx_lag_28','ratio_coupon_tx_lag_29',
               'tx_lag_2', 'tx_lag_3','tx_lag_4', 'tx_lag_5','tx_lag_6', 'tx_lag_7', 
                'tx_lag_8', 'tx_lag_9', 'tx_lag_10', 'tx_lag_11','tx_lag_12', 'tx_lag_13',
               'tx_lag_14', 'tx_lag_15', 'tx_lag_16','tx_lag_17', 'tx_lag_18','tx_lag_19',
                'tx_lag_20', 'tx_lag_21',
                 'var_tx_cancelled_lag_2', 'var_tx_cancelled_lag_3', 
                'var_tx_cancelled_lag_4', 'var_tx_cancelled_lag_5', 'var_tx_cancelled_lag_6', 
               'var_tx_cancelled_lag_7', 
               'var_tx_cancelled_lag_8', 'var_tx_cancelled_lag_9', 
                'var_tx_cancelled_lag_10', 'var_tx_cancelled_lag_11', 'var_tx_cancelled_lag_12', 
                'var_tx_cancelled_lag_13', 'var_tx_cancelled_lag_14','var_tx_cancelled_lag_15', 
                'var_tx_cancelled_lag_16', 'var_tx_cancelled_lag_17', 'var_tx_cancelled_lag_18',
                'var_tx_cancelled_lag_19', 'var_tx_cancelled_lag_20','var_tx_cancelled_lag_21', 
                'var_tx_cancelled_lag_22','var_tx_cancelled_lag_23', 'var_tx_cancelled_lag_24',
                'var_tx_cancelled_lag_25', 'var_tx_cancelled_lag_26','var_tx_cancelled_lag_27', 
                'var_tx_cancelled_lag_28', 'var_tx_cancelled_lag_29',
                'margin_lag_2', 'margin_lag_3', 'margin_lag_4', 'margin_lag_5',
                'margin_lag_6', 'margin_lag_7', 
                'margin_lag_8', 'margin_lag_9', 'margin_lag_10', 
               'is_holiday',
               'is_fourth_of_july',
               'var_30ds',
               'christmas_day',
               'new_year_day',
               'post_holiday',
               'day_of_the_dead'
              ]]

In [101]:
# Workaround para payers nuevos
# split_index = int(len(datos) * 0.5)

### TRAINING, VALIDATION AND TESTING PERIODS

In [102]:
# Train-validation-test
# ======================================================================================
fin_train = '2023-03-31'
fin_val = '2023-06-21'
print(
    f"Fechas entrenamiento : {datos.index.min()} --- {datos.loc[:fin_train].index.max()}  "
    f"(n={len(datos.loc[:fin_train])})"
)
print(
    f"Fechas validacion    : {datos.loc[fin_train:].index.min()} --- {datos.loc[:fin_val].index.max()}  "
    f"(n={len(datos.loc[fin_train:fin_val])})"
)
print(
    f"Fechas test          : {datos.loc[fin_val:].index.min()} --- {datos.index.max()}  "
    f"(n={len(datos.loc[fin_val:])})"
)


Fechas entrenamiento : 2021-01-01 00:00:00 --- 2023-03-31 00:00:00  (n=820)
Fechas validacion    : 2023-03-31 00:00:00 --- 2023-06-21 00:00:00  (n=83)
Fechas test          : 2023-06-21 00:00:00 --- 2023-12-18 00:00:00  (n=181)


### DEFINITION OF EXOGENOUS VARIABLES

In [103]:
exogenas=[
       'var_rate_lag_2','var_rate_lag_3','var_rate_lag_4', 'var_rate_lag_5',
       'var_rate_lag_6','var_rate_lag_7',
       'var_rate_lag_8','var_rate_lag_9',
               'var_rate_lag_10','var_rate_lag_11','var_rate_lag_12','var_rate_lag_13', 
                'var_rate_lag_14','var_rate_lag_15','var_rate_lag_16','var_rate_lag_17',
                'var_rate_lag_18','var_rate_lag_19','var_rate_lag_20','var_rate_lag_21',
                'var_rate_lag_22','var_rate_lag_23','var_rate_lag_24','var_rate_lag_25',
                'var_rate_lag_26','var_rate_lag_27','var_rate_lag_28','var_rate_lag_29', 
                'ratio_coupon_tx_lag_2', 
                'ratio_coupon_tx_lag_3', 'ratio_coupon_tx_lag_4', 'ratio_coupon_tx_lag_5', 
                 'ratio_coupon_tx_lag_6', 'ratio_coupon_tx_lag_7', 
                'ratio_coupon_tx_lag_8', 
               'ratio_coupon_tx_lag_9', 'ratio_coupon_tx_lag_10', 'ratio_coupon_tx_lag_11', 
                'ratio_coupon_tx_lag_12', 'ratio_coupon_tx_lag_13', 'ratio_coupon_tx_lag_14',
                'ratio_coupon_tx_lag_15','ratio_coupon_tx_lag_16','ratio_coupon_tx_lag_17', 
                'ratio_coupon_tx_lag_18','ratio_coupon_tx_lag_19','ratio_coupon_tx_lag_20',
                'ratio_coupon_tx_lag_21','ratio_coupon_tx_lag_22','ratio_coupon_tx_lag_23',
                'ratio_coupon_tx_lag_24','ratio_coupon_tx_lag_25','ratio_coupon_tx_lag_26',
                'ratio_coupon_tx_lag_27','ratio_coupon_tx_lag_28','ratio_coupon_tx_lag_29',
                'tx_lag_2', 'tx_lag_3','tx_lag_4', 'tx_lag_5','tx_lag_6', 'tx_lag_7', 
                'tx_lag_8', 'tx_lag_9', 'tx_lag_10', 'tx_lag_11','tx_lag_12', 'tx_lag_13',
                'tx_lag_14', 'tx_lag_15', 'tx_lag_16','tx_lag_17', 'tx_lag_18','tx_lag_19',
                'tx_lag_20', 'tx_lag_21',
                'var_tx_cancelled_lag_2', 'var_tx_cancelled_lag_3', 
                'var_tx_cancelled_lag_4', 'var_tx_cancelled_lag_5', 'var_tx_cancelled_lag_6', 
                'var_tx_cancelled_lag_7', 
                 'var_tx_cancelled_lag_8', 'var_tx_cancelled_lag_9', 
                'var_tx_cancelled_lag_10', 'var_tx_cancelled_lag_11', 'var_tx_cancelled_lag_12', 
                'var_tx_cancelled_lag_13', 'var_tx_cancelled_lag_14','var_tx_cancelled_lag_15', 
                'var_tx_cancelled_lag_16', 'var_tx_cancelled_lag_17', 'var_tx_cancelled_lag_18',
                'var_tx_cancelled_lag_19', 'var_tx_cancelled_lag_20','var_tx_cancelled_lag_21', 
                'var_tx_cancelled_lag_22','var_tx_cancelled_lag_23', 'var_tx_cancelled_lag_24',
                'var_tx_cancelled_lag_25', 'var_tx_cancelled_lag_26','var_tx_cancelled_lag_27', 
                'var_tx_cancelled_lag_28', 'var_tx_cancelled_lag_29',
                'margin_lag_2', 'margin_lag_3', 'margin_lag_4', 'margin_lag_5',
               'margin_lag_6', 'margin_lag_7', 
               'margin_lag_8', 'margin_lag_9',  'margin_lag_10', 
                'is_holiday',
               'is_fourth_of_july',
               'var_30ds',
               'christmas_day',
               'new_year_day',
               'post_holiday',
               'day_of_the_dead'
              ]

In [104]:
datos[exogenas] = datos[exogenas].fillna(0)

### SELECTION OF SIGNIFICANT EXOGENOUS VARIABLES

In [105]:
# Modelo SARIMAX con skforecast.Sarimax
# ==============================================================================
warnings.filterwarnings("ignore", category=UserWarning, message='Non-invertible|Non-stationary')
modelo = Sarimax(order=(2, 1, 1), seasonal_order=(2, 1, 1, 7))
modelo.fit(y=datos['amount'].loc[:fin_train], exog=datos[exogenas].loc[:fin_train] )
modelo.summary()
#warnings.filterwarnings("default")

/opt/conda/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                     
=========================================================================================
Dep. Variable:                            amount   No. Observations:                  820
Model:             SARIMAX(2, 1, 1)x(2, 1, 1, 7)   Log Likelihood              -11033.131
Date:                           Tue, 18 Jun 2024   AIC                          22320.263
Time:                                   12:39:06   BIC                          22917.099
Sample:                               01-01-2021   HQIC                         22549.379
                                    - 03-31-2023                                         
Covariance Type:                             opg                                         
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
var_rate_lag_2          -1.653e+05        nan        nan        nan         nan         nan
var_rate_lag_3           3.645e+05   1.37e+05      2.655      0.008    9.54e+04    6.34e+05
var_rate_lag_4           3.246e+05   6.72e+04      4.833      0.000    1.93e+05    4.56e+05
var_rate_lag_5           1.087e+06   1.31e+05      8.306      0.000    8.31e+05    1.34e+06
var_rate_lag_6            1.22e+06    1.7e+05      7.190      0.000    8.88e+05    1.55e+06
var_rate_lag_7           1.493e+06   1.69e+05      8.856      0.000    1.16e+06    1.82e+06
var_rate_lag_8           7.087e+05   1.32e+05      5.356      0.000    4.49e+05    9.68e+05
var_rate_lag_9           1.056e+06   1.21e+05      8.711      0.000    8.19e+05    1.29e+06
var_rate_lag_10          1.003e+06   1.18e+05      8.535      0.000    7.73e+05    1.23e+06
var_rate_lag_11          1.597e+06    1.2e+05     13.290      0.000    1.36e+06    1.83e+06
var_rate_lag_12          1.798e+06   1.18e+05     15.222      0.000    1.57e+06    2.03e+06
var_rate_lag_13            1.2e+06   1.38e+05      8.686      0.000    9.29e+05    1.47e+06
var_rate_lag_14          1.267e+06   1.45e+05      8.714      0.000    9.82e+05    1.55e+06
var_rate_lag_15          8.977e+05   1.14e+05      7.896      0.000    6.75e+05    1.12e+06
var_rate_lag_16          6.748e+05   1.36e+05      4.974      0.000    4.09e+05    9.41e+05
var_rate_lag_17          1.011e+06   8.45e+04     11.973      0.000    8.46e+05    1.18e+06
var_rate_lag_18          5.856e+05   1.09e+05      5.353      0.000    3.71e+05       8e+05
var_rate_lag_19          5.234e+05   1.19e+05      4.392      0.000     2.9e+05    7.57e+05
var_rate_lag_20         -9.876e+04   1.52e+05     -0.651      0.515   -3.96e+05    1.99e+05
var_rate_lag_21          8.358e+05   1.25e+05      6.672      0.000     5.9e+05    1.08e+06
var_rate_lag_22         -4191.5413   9.28e+04     -0.045      0.964   -1.86e+05    1.78e+05
var_rate_lag_23          6.598e+05   4.99e+04     13.214      0.000    5.62e+05    7.58e+05
var_rate_lag_24          3.714e+05   9.81e+04      3.787      0.000    1.79e+05    5.64e+05
var_rate_lag_25          7.246e+05   1.02e+05      7.082      0.000    5.24e+05    9.25e+05
var_rate_lag_26          6.256e+05   1.12e+05      5.605      0.000    4.07e+05    8.44e+05
var_rate_lag_27          2.796e+05   8.16e+04      3.426      0.001     1.2e+05     4.4e+05
var_rate_lag_28          5.584e+05   1.06e+05      5.248      0.000     3.5e+05    7.67e+05
var_rate_lag_29         -1.077e+05   6.32e+04     -1.706      0.088   -2.32e+05    1.61e+04
ratio_coupon_tx_lag_2    3.111e+05   1.65e+04     18.851      0.000    2.79e+05    3.43e+05
ratio_coupon_tx_lag_3    1.144e+06   1.03e+05     11.079      0.000    9.42e+05    1.35e+06
ratio_coupon_tx_lag_4    2.852e+05   3.94e+04      7.240      0.000    2.08e+05    3.62e+05
ratio_coupon_t

In [106]:

#with open(f'sarimax_summary_{payer}_7d_12_03.txt', 'w') as f:
    #f.write(modelo.summary().as_text())_

In [107]:
# Get summary table as string
tabla_resumen = modelo.summary().tables[1].as_html()
# Convert HTML string to a pandas DataFrame
df_resumen = pd.read_html(tabla_resumen, header=0, index_col=0)[0]
#Filter the DataFrame to keep only the rows that contain p values.
df_valores_p = df_resumen[df_resumen.index.str.contains('P>|z|')]
#Define the significance threshold
umbral_significancia = 0.05
# Filter out the rows where the p-value is less than the significance threshold.
variables_significativas = df_valores_p[df_valores_p['P>|z|'] < umbral_significancia]
indices_variables_significativas = variables_significativas.index.tolist()
# Convert indexes to a list of characters
exogenas_total = list(map(str, indices_variables_significativas))
#Filtered exogenous list
exogenas = [var for var in exogenas_total if not var.startswith(('ar.', 'ma.', 'sigma'))]
# Show filtered exogenous list
print(exogenas)

['var_rate_lag_3', 'var_rate_lag_4', 'var_rate_lag_5', 'var_rate_lag_6', 'var_rate_lag_7', 'var_rate_lag_8', 'var_rate_lag_9', 'var_rate_lag_10', 'var_rate_lag_11', 'var_rate_lag_12', 'var_rate_lag_13', 'var_rate_lag_14', 'var_rate_lag_15', 'var_rate_lag_16', 'var_rate_lag_17', 'var_rate_lag_18', 'var_rate_lag_19', 'var_rate_lag_21', 'var_rate_lag_23', 'var_rate_lag_24', 'var_rate_lag_25', 'var_rate_lag_26', 'var_rate_lag_27', 'var_rate_lag_28', 'ratio_coupon_tx_lag_2', 'ratio_coupon_tx_lag_3', 'ratio_coupon_tx_lag_4', 'ratio_coupon_tx_lag_5', 'ratio_coupon_tx_lag_6', 'ratio_coupon_tx_lag_8', 'ratio_coupon_tx_lag_9', 'ratio_coupon_tx_lag_10', 'ratio_coupon_tx_lag_11', 'ratio_coupon_tx_lag_12', 'ratio_coupon_tx_lag_15', 'ratio_coupon_tx_lag_16', 'ratio_coupon_tx_lag_17', 'ratio_coupon_tx_lag_18', 'ratio_coupon_tx_lag_19', 'ratio_coupon_tx_lag_20', 'ratio_coupon_tx_lag_21', 'ratio_coupon_tx_lag_22', 'ratio_coupon_tx_lag_26', 'ratio_coupon_tx_lag_27', 'tx_lag_2', 'tx_lag_3', 'tx_lag_4', '

/tmp/ipykernel_20/159567767.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_resumen = pd.read_html(tabla_resumen, header=0, index_col=0)[0]


In [108]:
# Supongamos que `modelo` es tu modelo previamente ajustado
# Obtener la tabla resumen como cadena de HTML
tabla_resumen = modelo.summary().tables[1].as_html()


In [ ]:
#We open the file in write mode, with the 'append' option to add content to the end of the file
#with open(f'sarimax_summary_{payer}_7d_8lags.txt', 'a') as f:
with open(f'sarimax_summary_{payer}_2d.txt', 'a') as f:
    # Escribir el contenido de la variable 'exogenas' en el archivo
    f.write('\n\nContenido de la variable "exogenas":\n')
    f.write(str(exogenas))

### SEARCH THE BEST MODEL 

In [ ]:
# Auto arima: Selection of the best model according to Akaike's criteria
# ==============================================================================
resultados_autoarima = auto_arima(
            y                 = datos['amount'].loc[:fin_val],
            exog              = datos[exogenas].loc[:fin_val],
            start_p           = 0, #Start value of the number of autoregressions
            start_q           = 0, #Moving average component order start value
            max_p             = 7, #Maximum order of the autoregressive component
            max_q             = 7, #Maximum order of the moving average component
            seasonal          = True, # It is indicated that the variable has seasonal behavior
            m                 = 7, # Weekly periodicity of seasonality
            max_d             = 2, # Maximum order of integration of the original series
            max_D             = 2, # Maximum order of differentiation of the seasonal series
            trace             = True, #Indicates that all steps in the process of searching for the best model are displayed.
            error_action      = 'ignore',
            suppress_warnings = True, 
            stepwise          = True #Iterate from a simple model to a more complex model by adding autoregressive orders, moving averages and integration.
        )
opt_order_2 = resultados_autoarima.order
opt_seasonal_order_2 = resultados_autoarima.seasonal_order

### MODEL BACKTESTING 

In [ ]:
forecaster_autoarima = ForecasterSarimax(
                 regressor=Sarimax(order=opt_order_2, seasonal_order=opt_seasonal_order_2)
             )
metrica_autoarima, predicciones_autoarima = backtesting_sarimax(
                                forecaster            = forecaster_autoarima,##Model fitted with parameters of the best model
                                y                     = datos['amount'],##The target variable is indicated
                                exog                  = datos[exogenas],##Selected exogenous variables
                                initial_train_size    = len(datos.loc[:fin_val]),##Test start period
                                steps                 = 2, ##Number of days to predict
                                metric                = 'mean_absolute_percentage_error',##evaluation metrics
                                refit                 = False,##Indicates not to update the model as it incorporates information
                                n_jobs                = 'auto',
                                suppress_warnings_fit = True,
                                verbose               = False,#Indicates not to show the entire iterative process
                                show_progress         = True#Indicates the degree of progress of the process.
                            )
# Prediction
nueva_fecha = datetime.strptime(fin_val, '%Y-%m-%d') + timedelta(days=1)
predicciones_autoarima = predicciones_autoarima['pred']

# Combining the two DataFrames into one
resultados_predichos_autoarima= pd.DataFrame({'date': predicciones_autoarima.index, 'valor_real': datos['amount'].loc[nueva_fecha:], 'valor_predicho': predicciones_autoarima})
# MAPE calculation
resultados_predichos_autoarima['mape'] = abs((resultados_predichos_autoarima['valor_real'] - resultados_predichos_autoarima['valor_predicho']) / resultados_predichos_autoarima['valor_real']) * 100
resultados_predichos_autoarima['error_abs'] = abs((resultados_predichos_autoarima['valor_real'] - resultados_predichos_autoarima['valor_predicho']))
# Calculate the average of the 'mape' column
promedio_mape_autoarima = resultados_predichos_autoarima['mape'].mean()
promedio_error_abs_autoarima = resultados_predichos_autoarima['error_abs'].mean()
# Print the average
print("Promedio de la columna 'mape' - 2 dias:", promedio_mape_autoarima)
print("Promedio de la columna 'error_abs' - 2 dias:", promedio_error_abs_autoarima)

### FIT

In [ ]:
datos=datos.loc[datos.index<'2023-12-19']

In [ ]:
datos.index.max()

In [ ]:
forecaster_autoarima.fit(y=datos['amount'],exog=datos[exogenas] ,suppress_warnings=True)

In [ ]:
forecaster_autoarima

In [ ]:
##Influential variables##
feat_imp=forecaster_autoarima.get_feature_importances()
feat_imp

In [ ]:
pd.set_option('display.max_rows', None)
feat_imp.sort_values(by='importance', ascending=False)

In [ ]:
with open(f'sarimax_summary_{payer}_2d.txt', 'a') as f:
    # Escribir el contenido de la variable 'exogenas' en el archivo
    f.write('\n\nContenido de la variable "exogenas":\n')
    f.write(str(forecaster_autoarima))

In [ ]:
modelo_final = forecaster_autoarima
resultados_predichos = resultados_predichos_autoarima
mape_final = promedio_mape_autoarima

In [ ]:
# Create a folder called 'results' if it does not exist
folder_name = 'top_payers_abt_v3update/'
carpeta_resultados = path2+folder_name+payer+'/'
print(carpeta_resultados)

try:
    client.put_object(Bucket=bucket_name, Key=(carpeta_resultados+'/'))
    print(f"Carpeta '{carpeta_resultados}' creada en el bucket '{bucket_name}'.")
except Exception as e:
    print(f"Error al crear la carpeta: {e}")
    

### SAVES THE RESULTS IN AN EXCEL FILE AND THE MODEL IN A PICKLE FILE

In [ ]:

#WE SAVE RESULTS IN EXCEL

nombre_archivo_excel = os.path.join(carpeta_resultados, 'predicciones_test_2d_'+payer+'_2d'+'v2'+'.xlsx')
# Guarda el DataFrame en un objeto BytesIO como archivo Excel
#excel_buffer = BytesIO()
#resultados_predichos.to_excel(excel_buffer, index=False)
# Save the DataFrame to an XLSX file in S3 using AWS Data Wrangler
wr.s3.to_excel(resultados_predichos, f"s3://{bucket_name}/{nombre_archivo_excel}", index=False)


nombre_archivo_pickle = os.path.join(carpeta_resultados, 'MODEL_2d_'+payer+'_2d'+'v2'+'.pkl')
# Save the model in a temporary pickle file
modelo_temp_file = 'MODEL_2d_'+payer+'.pkl'
joblib.dump(modelo_final, modelo_temp_file)

# Upload the pkl file to S3
try:
    wr.s3.upload(modelo_temp_file, f"s3://{bucket_name}/{nombre_archivo_pickle}")
    print(f"Modelo guardado como pkl en S3: {nombre_archivo_pickle}")
    os.remove(modelo_temp_file) # Elimina el temporal
except Exception as e:
    print(f"Error al subir el modelo a S3: {e}")


In [ ]:
#TXT
file_name = f'sarimax_summary_{payer}_2d.txt'
ruta_txt = os.path.join(carpeta_resultados, file_name)
# Cargar el archivo en S3
s3 = boto3.resource("s3")
s3.Bucket(bucket_name).upload_file(file_name, ruta_txt)
os.remove(file_name) # Elimina el temporal

In [ ]:
# Celda de finalización
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Tiempo de ejecución total: {elapsed_time} segundos")